In [1]:
import spacy
import json
import socket
import time
from time import mktime
from datetime import datetime, date

import os

import psutil
import requests
from pytz import timezone
from weather import Unit, Weather
import multiprocessing

import random
import parsedatetime as pdt

import json
import re
import string
import unicodedata
from io import open

import nltk
import numpy as np
import pandas as pd
from keras.layers import LSTM, Dense, Embedding, Input
from keras.models import Model, Sequential
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


nltk.download('stopwords')

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\home\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:


class DialogModel:
    def __init__(self, encoder_input_data,
                 decoder_input_data, decoder_target_data):
        self.encoder_input_data = encoder_input_data
        self.decoder_input_data = decoder_input_data
        self.decoder_target_data = decoder_target_data

        # intialize attributes for encoder, decoder model
        self.encoder_inputs = None
        self.decoder_inputs = None
        self.decoder_outputs = None
        self.decoder_latent_dim = None
        self.history = None

        # initializing attributes for inference model
        self.encoder_model = None
        self.decoder_model = None
        self._decoder = None
        self._decoder_dense = None
        self.inference_ready = False

    def encoder(self, input_len, latent_dim=128, activation='softmax'):
        self.encoder_inputs = Input(shape=(None, input_len))
        encoder = LSTM(latent_dim, activation=activation, return_state=True)
        _, state_h, state_c = encoder(self.encoder_inputs)
        self.encoder_states = [state_h, state_c]

    def decoder(self, output_len, latent_dim=128, activation='tanh'):
        self.decoder_latent_dim = latent_dim
        self.decoder_inputs = Input(shape=(None, output_len))
        self._decoder = LSTM(
            self.decoder_latent_dim, activation=activation,
            return_sequences=True, return_state=True)
        decoder_outputs, _, _ = \
            self._decoder(self.decoder_inputs,
                          initial_state=self.encoder_states)
        self._decoder_dense = Dense(output_len, activation='softmax')
        self.decoder_outputs = self._decoder_dense(decoder_outputs)

    def train(self, optimizer='rmsprop', loss='categorical_crossentropy',
              batch_size=64, epochs=100, validation=0.0, summary=False, name=''):
        model = Model(
            [self.encoder_inputs, self.decoder_inputs], self.decoder_outputs)

        if summary:
            model.summary()

        model.compile(optimizer, loss, metrics=['mae', 'categorical_accuracy'])
        self.history = model.fit(
            [self.encoder_input_data, self.decoder_input_data],
            self.decoder_target_data, batch_size=batch_size,
            epochs=epochs, validation_split=validation)
        
        model.save(name + '.dialog.model.h5')
        
    def load(self, name = ''):
        model = Model(
            [self.encoder_inputs, self.decoder_inputs], self.decoder_outputs)
        model.load_weights(name + '.dialog.model.h5')

    def build_inference_model(self):
        self.encoder_model = Model(self.encoder_inputs, self.encoder_states)

        latent_dim = self.decoder_latent_dim
        decoder_state_input_h = Input(shape=(latent_dim,))
        decoder_state_input_c = Input(shape=(latent_dim,))
        decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
        decoder_outputs, state_h, state_c = self._decoder(
            self.decoder_inputs, initial_state=decoder_states_inputs)
        decoder_states = [state_h, state_c]
        decoder_outputs = self._decoder_dense(decoder_outputs)
        self.decoder_model = Model(
            [self.decoder_inputs] + decoder_states_inputs,
            [decoder_outputs] + decoder_states)

        self.inference_ready = True

    def decode(self, input_sequence, output_word_model, max_sentence_len):
        if not self.inference_ready:
            self.build_inference_model()

        state_values = self.encoder_model.predict(input_sequence)
        output_len = output_word_model.n_words
        target_seq = np.zeros((1, 1, output_len))
        target_seq[0, 0, output_word_model.word2index['SOS']] = 1
        stop_cond = False
        decoded_sent = []
        confidence = 0
        while not stop_cond:
            output_tokens, h, c = \
                self.decoder_model.predict([target_seq] + state_values)
            sampled_token_index = np.argmax(output_tokens[0, -1, :])
            word = output_word_model.index2word[sampled_token_index]
            if word == 'EOS' or len(decoded_sent) > max_sentence_len:
                stop_cond = True
            else:
                confidence += max(output_tokens[0, -1, :])
                decoded_sent.append(word)
            target_seq = np.zeros((1, 1, output_len))
            target_seq[0, 0, output_word_model.word2index[word]] = 1
            state_values = [h, c]
        return " ".join(decoded_sent), confidence/len(decoded_sent)


In [3]:


def IntentModel(model):
    model = model.lower()
    if model == "onehot":
        return OneHotModel
    elif model == "embeddings":
        return EmbeddingsModel
    else:
        print("{} does not exist".format(model))
        return None


class BaseIntentModel:
    def __init__(self):
        self.x_train = None
        self.y_train = None
        self.model = None
        self.history = None

    def train(self, optimizer='rmsprop', loss='categorical_crossentropy',
              batch_size=5, epochs=100, validation=0.0, summary=False, name = ''):
        if summary:
            self.model.summary()

        self.model.compile(optimizer, loss, metrics=['mae', 'categorical_accuracy'])
        self.history = self.model.fit(
            self.x_train, self.y_train,
            batch_size=batch_size, epochs=epochs,
            validation_split=validation)
        if not name == '':
          self.model.save_weights(name + '.intent.model.h5')
        
    def load(self, name):
      self.model.load_weights(name + '.intent.model.h5')
      
    def save(self, name):
      self.model.save_weights(name + '.intent.model.h5')

    def decode(self, input_sequence, output_word_model):
        output_tokens = self.model.predict(input_sequence)
        token_index = np.argmax(output_tokens[0])
        intent = output_word_model.index2word[token_index]
        confidence = max(output_tokens[0])
        return intent, confidence


class OneHotModel(BaseIntentModel):
    def __init__(self, x_train, y_train,
                 input_len, output_len,
                 latent_dim=128, activation='tanh'):
        self.x_train = x_train
        self.y_train = y_train
        self.model = \
            self.build_model(input_len, output_len, latent_dim, activation)

    def build_model(self, input_len, output_len, latent_dim, activation):
        model = Sequential()
        model.add(
            LSTM(latent_dim, activation=activation,
                 input_shape=(None, input_len)))
        model.add(Dense(36, activation = 'relu'))
        model.add(Dense(output_len, activation='softmax'))
        return model


class EmbeddingsModel(BaseIntentModel):
    def __init__(self, x_train, y_train,
                 input_dim, output_dim,
                 latent_dim=128, activation='tanh'):
        self.x_train = x_train
        self.y_train = y_train
        self.model = \
            self.build_model(input_dim, output_dim, latent_dim, activation)

    def build_model(self, input_dim, output_dim, latent_dim, activation):
        model = Sequential()
        model.add(Embedding(input_dim, output_dim,))
        model.add(
            LSTM(latent_dim, activation=activation,
                 input_shape=(None, input_dim)))
        model.add(Dense(36, activation = 'relu'))
        model.add(Dense(output_dim, activation='softmax'))
        return model


In [4]:

PAD = "PAD"
SOS = "SOS"
EOS = "EOS"
UNK = "UNK"


class WordModel:
    def __init__(self, name):
        self.name = name
        self.word2index = {PAD: 0, SOS: 1, EOS: 2, UNK: 3}
        self.word2count = {}
        self.index2word = {0: PAD, 1: SOS, 2: EOS, 3: UNK}
        self.n_words = 4  # count for default tokens

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1


# helper functions to prepocess
def unicode_to_ascii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


def normalize_string(s):
    s = unicode_to_ascii(s.lower().rstrip())
    s = re.sub(r"([!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.=!?{}:]+", r" ", s)
    if s[-1] == ".":
        s = s[:-1]+" ."
    return s


def filter_pair(p):
    """" filter sequence by MAX_LENGTH """
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH


def filter_pairs(pairs):
    """ wrapper that filter sequence pairs by MAX_LENGTH """
    return [pair for pair in pairs if filter_pair(pair)]


def get_data_pairs_from_json(_input, _output, filename):
    """ """
    with open(filename, 'r') as f:
        data = json.load(f)
    pairs = []
    for pair in data:
        pairs.append(
            [normalize_string(pair[_input]), normalize_string(pair[_output])])
    input_word_model = WordModel(_input)
    output_word_model = WordModel(_output)
    return input_word_model, output_word_model, pairs


def prepare_json_data(_input, _output, filename, max_length=15, name = ""):
    """ prepares data for processing. Expect json structure to follow...

            {
                "input": "input string"
                "output": "output string"
            }

        Args:
            _input (str): string of the key for model input
            _output (str): string of the key for model output
            filename (str): string of absolute path of file
            max_length (int): Maximum length of the sequence

        Returns:
            input_word_model (WordModel): object containing input defintiions
            output_word_model (WordModel): object containing output definitions
            pairs (list<str>): [input, output] sentences
    """
    global MAX_LENGTH
    MAX_LENGTH = max_length

    input_word_model, output_word_model, pairs = \
        get_data_pairs_from_json(_input, _output, filename)
    if name == "":
        print("READ %s sentence pairs" % len(pairs))
    else:
        print(name, "\n", "\t", "READ %s sentence pairs" % len(pairs))
    pairs = filter_pairs(pairs)
    if name == "":
        print("Trimmed to %s sentence pairs" % len(pairs))
        print("Counting words...")
    else:
        print("\tTrimmed to %s sentence pairs" % len(pairs))
        print("\tCounting words...")
    for pair in pairs:
        input_word_model.add_sentence(pair[0])
        output_word_model.add_sentence(pair[1])
    if name == "":
        print("Counted Words:")
        print(input_word_model.name, input_word_model.n_words)
        print(output_word_model.name, output_word_model.n_words)
    else:
        print("\tCounted Words:")
        print("\t", input_word_model.name, input_word_model.n_words)
        print("\t", output_word_model.name, output_word_model.n_words)
    return input_word_model, output_word_model, pairs


def pad_sequence(sequence, seq_len):
    pad_len = (seq_len - 2) - len(sequence.split())
    pads = [PAD for i in range(pad_len)]
    _seq = " ".join([SOS, *sequence.split()[:seq_len-2], EOS, *pads])
    assert len(_seq.split()) == seq_len
    return _seq


def pad_sequences(sequences, seq_len):
    """ adds PAD, SOS, EOS, UNK to sequence """
    padded_seqs = []
    for seq in sequences:
        _seq = pad_sequence(seq, seq_len)
        padded_seqs.append(_seq.split())
    return padded_seqs


def integer_encode(sequences, word_model, seq_len):
    """ returns nparray index encoded sequences """
    encoded_list = np.zeros((len(sequences), seq_len))
    for i, seq in enumerate(sequences):
        for j, word in enumerate(seq):
            if word not in word_model.word2index:
                word = UNK
            encoded_list[i, j] = word_model.word2index[word]
    return encoded_list


def one_hot_encode(sequences, word_model, seq_len):
    """ returns nparray with one hot encoded sequences """
    one_hot = np.zeros((len(sequences), seq_len, word_model.n_words))
    for i, seq in enumerate(sequences):
        for j, word in enumerate(seq):
            if word != PAD:
                if word not in word_model.word2index:
                    word = UNK
                one_hot[i, j, word_model.word2index[word]] = 1
    return one_hot


def one_hot_encode_target(sequences, word_model, seq_len):
    """" encode target with an offset of 1 for decoder in seq2seq """
    one_hot = np.zeros((len(sequences), seq_len, word_model.n_words))
    for i, seq in enumerate(sequences):
        for j, word in enumerate(seq):
            if word != PAD and j > 0:
                one_hot[i, j-1, word_model.word2index[word]] = 1
    return one_hot


In [5]:

class Actions:
  
  def __init__(self):
    self.Functions = {}
    self.NullFunctions = {}
    return
  
  def Add(self, Name, Function = None):
    if Function == None:
      self.NullFunctions[Name] = None
    self.Functions[Name] = Function
    #print(Name, ' : ', Function)
    return
  
  def Func(self):
    return self.Functions.items()
  
  def Print(self):
    for key, value in self.Functions.items():
      print('[', key , ' : ', value, ']')
    return
  
  def ExecuteAction(self, intent, entities):
    try:
      if not len(entities.items()) == 0:
        return self.Functions[intent](entities)
      return self.Functions[intent]()
    except:
      # Add Exception, Error => log For why.did.skil.fail
      return


  def Mix(self, Entities, dialog):
    for key, value in Entities.items():
      r = key
      r = r.replace('_', ' ')
      r = "{" + r + "}"
      dialog = dialog.replace(r, value)
    return dialog
  
  def GetFeedbackDialog(self, intent, entities, dialog):
    try:
      if intent in self.NullFunctions.keys():
        f = open('error.log', 'a')
        f.writelines('\n' + intent + '|actions.exceptions.NoActionFoundError')
        f.close()
        return ''.join(['Oops! looks like you are out of luck... ' , 
        'This project is just for educational purpose and is a demo,' ,
        ' not a production ready app. Thus some of the capabilities are',
        ' not yet added. Please add them manually or contact the' ,
        ' developer at GitHub : arnavdas88 '])

      if intent in self.Functions.keys():
        if len(entities.items()) == 0:
          entities = self.Functions[intent]()
        else:
          entities = self.Functions[intent](entities)
      else:
        return dialog
    except:
      # Add Exception, Error => log For why.did.skil.fail
      error = str(sys.exc_info()[0]).replace('<class \'','').replace('\'>', '')
      f = open('error.log', 'a')
      f.writelines('\n' + intent + '|' + error)
      f.close()
      return "Oops! " + error + " occured."
    '''
    print('\n\t', entities.items(), '\n')
    '''

    for key, value in entities.items():
      r = key
      r = r.replace('_', ' ')
      r = "{" + r + "}"
      value = str(value)
      #print('[', key, ': ', value, ']')
      dialog = dialog.replace(r, value)
    return dialog



In [6]:
class NER:
  def __init__(self, lang = 'en'):
    self.nlp = spacy.load('en')
    self.PropertyDiscription = {'GPE' : 'location', 'TIME' : 'time', 'DATE' : 'date'}
    self.replaceWords = {}
    return
  
  def GetFor(self, text = ''):
    doc = self.nlp(text)
    '''
    for ent in doc.ents:
        print(ent, ent.label_, [token.dep_ for token in ent])
    '''
    return doc.ents
  
  def AddCustomReplaceWords(self, words, _class):
    for word in words:
      self.replaceWords[word] = _class
    return

  def GetAllEntities(self, text = ''):
    doc = self.nlp(text)
    entities = {self.replaceWords[word] : word for word in text.split(' ') if word in self.replaceWords.keys()}
    for key, value in entities.items():
      r = key
      r = r.replace('_', ' ')
      r = "{" + r + "}"
      value = str(value)
      text = text.replace(r, value)
    entities.update({self.PropertyDiscription[ent.label_] : str(ent) for ent in doc.ents if ent.label_ in self.PropertyDiscription.keys()})
    for key, value in entities.items():
      r = key
      r = r.replace('_', ' ')
      r = "{" + r + "}"
      value = str(value)
      text = text.replace(r, value)
    return text, entities

  def train(self):
    return


In [7]:

def get_ip(inputs = {}):
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    try:
        # doesn't even have to be reachable
        s.connect(('10.255.255.255', 1))
        IP = s.getsockname()[0]
    except:
        IP = '127.0.0.1'
    finally:
        s.close()
    return {'ip' : IP}

def get_date(inputs = {}):
  today = date.today()
  month = today.strftime('%B')
  day = today.day

  if (3 < day < 21) or (23 < day < 31):
    suffix = 'th'
  else:
    suffixes = {1: 'st', 2: 'nd', 3: 'rd'}
    suffix = suffixes[day % 10]
  
  return {'day' : day, 'day_suffix' : suffix, 'month' : month}

def get_time(inputs = {'location' : ''}):

  place = ''
  if not len(inputs.items()) == 0:
    place = inputs['location']
  
  if place == '':
    ts = time.time()
    st = datetime.fromtimestamp(ts).strftime('%I %M %p')
    tTime = st.split(' ')
    t = str(int(tTime[0])) + ' ' + tTime[1] + ' ' + tTime[2]
    return {'time' : t}

  from tzwhere import tzwhere
  place = place.replace(' ', '+')
  search_details = 'https://geocoder.api.here.com/6.2/geocode.json?app_id=cb870QP7ugZzb4tWAcKI&app_code=d4xPcsr4E8fj_rzUNhkXgQ&searchtext=' + place

  resp = requests.get(url=search_details, params='')
  data = resp.json()
  

  tz = tzwhere.tzwhere()
  lat = data['Response']['View'][0]['Result'][0]['Location']['DisplayPosition']['Latitude']
  lon = data['Response']['View'][0]['Result'][0]['Location']['DisplayPosition']['Longitude']

  
  timeZoneStr = tz.tzNameAt(lat, lon)
  timeZoneObj = timezone(timeZoneStr)
  ts = datetime.now(timeZoneObj).strftime('%H:%M:%S')
  return {'time' : ts}

def get_cpu_max_process(inputs = {}):
  name = ''
  cpu = '0.0'
  CPU_CORE = float(multiprocessing.cpu_count())
  for _ in range(4):
      pro = {float(proc.cpu_percent(interval=None) / CPU_CORE) : str(proc.name()) for proc in psutil.process_iter()}
      cpu = max(pro.keys(), key=float)
      name = pro[cpu]
      #if not _name == 'pythonw.exe':
      #cpu = _cpu
      #name = _name
      #print(name, ' : ', cpu, '%')
  return {'name' : name, 'percent' : str(cpu) + '%'}

def get_weather_current(inputs = {'location' : ''}):
  
  location = ''
  if not len(inputs.items()) == 0:
    location = inputs['location']

  weather = Weather(unit=Unit.CELSIUS)
  
  if location == '':
    r = requests.get(url = 'https://ipinfo.io/loc', params = '').text
    lookup = weather.lookup_by_latlng(r.split(',')[0], r.split(',')[1])
  else:
    lookup = weather.lookup_by_location(location)
    
  condition = lookup.condition
  location = lookup.location
  forecast = lookup.forecast
  return {'condition' : condition.text, 'temp_current' : condition.temp, 'scale' : 'C', 'diff_location' : location.city + ', ' + location.country, 'temp_max' : forecast[0].high, 'temp_min' : forecast[0].low }

def get_weather_next_day(inputs = {'location' : ''}):
  
  location = ''
  if not len(inputs.items()) == 0:
    location = inputs['location']

  weather = Weather(unit=Unit.CELSIUS)
  
  if location == '':
    r = requests.get(url = 'https://ipinfo.io/loc', params = '').text
    lookup = weather.lookup_by_latlng(r.split(',')[0], r.split(',')[1])
  else:
    lookup = weather.lookup_by_location(location)
    
  condition = lookup.condition
  location = lookup.location
  forecast = lookup.forecast
  return {'condition' : forecast[1].text, 'diff_location' : location.city + ', ' + location.country, 'temp_max' : forecast[1].high, 'temp_min' : forecast[1].low }

def set_alarm(inputs = {'time' : '', 'date' : ''}):
  
  if not 'time' in inputs.keys():
    time = datetime.now()
  else:
    time = inputs['time'].replace('next', '+1')
  
  if not 'date' in inputs.keys():
    date = datetime.today().strftime('%m/%d/%Y')
  else:
      if inputs['date'] == 'today':
        date = datetime.today().strftime('%m/%d/%Y')
      else:
        date = inputs['date']
  name = 'alarm'
  str_dt = time + ' ' + date
  p = pdt.Calendar()
  _datetime = p.parse(str_dt)
  dt = datetime.fromtimestamp(mktime(_datetime[0]))
  time = str(dt.time())
  date = str(dt.strftime('%m/%d/%Y'))
  name = name + '|' + time + '|' + date
  name = name.replace(':', '').replace('/', '').replace('|', '')
  f = open('alarmList.txt', 'a')
  f.write(name + '|' + time + '|' + date + '\n')
  f.close()
  cwd = os.getcwd()
  print(cwd)
  comd = 'schtasks /create /tn \"' + name + '\" /tr ' + cwd + '\\alarm.bat /sc once /st ' + str(time) + ' /sd ' + str(date)
  os.system(comd)
  return {'weekday' : dt.strftime('%A'), 'hour' : dt.strftime('%I'), 'minute' : dt.strftime('%M'), 'am_pm' : dt.strftime('%p')}

def remove_alarm(inputs = {}):
    f = open('alarmList.txt', 'r')
    lines = f.readlines()
    f.close()
    names = [line.split('|')[0] for line in lines]
    template = 'schtasks /delete /tn '
    [os.system(template + name + ' /F') for name in names]
    f = open('alarmList.txt', 'w')
    f.write('')
    f.close()
    return {}

def list_alarm(inputs = {}):
    f = open('alarmList.txt', 'r')
    lines = f.readlines()
    f.close()
    names = [[line.split('|')[0], line.split('|')[1] + ' ' + line.split('|')[2]] for line in lines]
    p = pdt.Calendar()
    datetimelist = [(name[0], datetime.fromtimestamp(mktime(p.parse(name[1])[0]))) for name in names]
    datetimelist = [Adatetime for (name, Adatetime) in datetimelist if Adatetime > datetime.now()]
    datetimelist = [dt.strftime('%A, %d. %B %Y at %I:%M%p') for dt in datetimelist]
    datetimelist = ', '.join(datetimelist)
    if datetimelist == '':
        return {'alarms' : 'No Alarms Found'}
    return {'alarms' : datetimelist}

def why_skill_failed(inputs = {'skill' : ''}):
    f = open('error.log', 'r')
    lines = f.readlines()
    if lines == []:
        return {'skill' : '', 'error' : 'No Error Found'}
    if not inputs['skill'] == '':
        lines = [line for line in lines if line.split('|')[0] == inputs['skill']]
    error = lines[len(lines) - 1]
    f.close()
    return {'skill' : error.split('|')[0], 'error' : error.split('|')[1]}

def get_jokes(inputs = {}):
    joke=open('jokes.json').read()
    joke = json.loads(joke)
    jcount = joke.__len__()
    joke = str(joke[random.randint(0, jcount)]['body'])
    return {'joke' : joke}



In [ ]:

#NER
emodel = NER()

#Dialog
MAX_SEQ_LEN_IN = 15
MAX_SEQ_LEN_OUT = 30

dinput_word_model, doutput_word_model, dpairs = \
    prepare_json_data('input', 'output', './dialog.json', MAX_SEQ_LEN_OUT, name = "Dialog")

dinput_seqs = [pair[0] for pair in dpairs]
doutput_seqs = [pair[1] for pair in dpairs]

#print([input_seq for input_seq in dinput_seqs])

dpadded_input = pad_sequences(dinput_seqs, MAX_SEQ_LEN_IN)
dpadded_output = pad_sequences(doutput_seqs, MAX_SEQ_LEN_OUT)

done_hot_input = one_hot_encode(dpadded_input, dinput_word_model, MAX_SEQ_LEN_IN)
done_hot_output = one_hot_encode(dpadded_output, doutput_word_model, MAX_SEQ_LEN_OUT)

done_hot_target = one_hot_encode_target(dpadded_output, doutput_word_model, MAX_SEQ_LEN_OUT)


dmodel = DialogModel(done_hot_input, done_hot_output, done_hot_target)
dmodel.encoder(293)
dmodel.decoder(152)
dmodel.load(name = 'Full')

print("\n")

#Intent

MAX_SEQ_LEN = 25
iinput_word_model, ioutput_word_model, ipairs = \
    prepare_json_data('input', 'output', './intents.json', MAX_SEQ_LEN_IN, name = "Intent")

input_seqs = [pair[0] for pair in ipairs]
intent = [pair[1] for pair in ipairs]

ipadded_input = pad_sequences(input_seqs, MAX_SEQ_LEN_OUT)

ione_hot_input = one_hot_encode(ipadded_input, iinput_word_model, MAX_SEQ_LEN_IN)
ione_hot_output = one_hot_encode([intent], ioutput_word_model, len(intent))[0]


Imodel = IntentModel("onehot")
Imodel = Imodel(ione_hot_input, ione_hot_output, 244, 53)

Imodel.load(name = 'Full')

actionJar = Actions()



emodel.AddCustomReplaceWords(list(set([intent[1] for intent in ipairs if '{'+intent[1]+'}' not in intent[0].split(' ')])), 'skill')

# Adding the Actions to the intents...

# actionJar.Add(INTENT, FUNCTION)

actionJar.Add('ip', get_ip)
actionJar.Add('time', get_time)
actionJar.Add('date', get_date)
actionJar.Add('cpu.max.process', get_cpu_max_process)
actionJar.Add('weather.current', get_weather_current)
actionJar.Add('weather.next.hour', get_weather_current)
actionJar.Add('weather.next.day', get_weather_next_day)
actionJar.Add('why.did.skill.fail', why_skill_failed)
actionJar.Add('alarm.remove', remove_alarm)
actionJar.Add('alarm.list', list_alarm)
actionJar.Add('alarm.set', set_alarm)
actionJar.Add('joke', get_jokes)

# Actions that do needs but do not have an associated function yet
actionJar.Add('say')
actionJar.Add('pair')
actionJar.Add('spell')
actionJar.Add('show')
actionJar.Add('open.path')
actionJar.Add('ask')
actionJar.Add('remove.all')
actionJar.Add('add')
actionJar.Add('open.palce')
actionJar.Add('update.all')
actionJar.Add('remove')
actionJar.Add('update')
actionJar.Add('math')
actionJar.Add('alarm.stop')
actionJar.Add('pandora.play')
actionJar.Add('pandora.next')
actionJar.Add('pandora.end')
actionJar.Add('pandora.stop')
actionJar.Add('cpu.total.usage')
actionJar.Add('cpu.usage.application')
actionJar.Add('mem.free')
actionJar.Add('record.end')
actionJar.Add('mem.usage.application')
actionJar.Add('mem.max.process')
actionJar.Add('record.begin')
actionJar.Add('playback.begin')
actionJar.Add('mem.total')
actionJar.Add('mem.used')
actionJar.Add('how.do.i.activate')

print("All modules loaded successfully...")
print("Closed domain task oriented bot operational...")
print("\n")

try:
    while True:
        _input = input("You : ")
        ipadded_input = [pad_sequence(_input.lower(), MAX_SEQ_LEN_IN).split()]
        ione_hot = one_hot_encode(ipadded_input, iinput_word_model, MAX_SEQ_LEN_IN)
        intent, iconfidence = Imodel.decode(ione_hot, ioutput_word_model)
        print("\n\t intent: ", intent, iconfidence)
        
        
        _input, entities = emodel.GetAllEntities(_input)
        print('\t entity:', entities)
        


        '''
        print(actionJar.ExecuteAction(intent, entities))
        '''

        _input = ':' + intent + ': ' + _input
        #print('\t', _input)
        
        dpadded_input = [pad_sequence(_input, len(_input.split(' '))).split()]
        done_hot = one_hot_encode(dpadded_input, dinput_word_model, MAX_SEQ_LEN_IN)
        dprediction, dconfidence = dmodel.decode(done_hot, doutput_word_model, MAX_SEQ_LEN_OUT)
        #print("\t\tresponse: ", dprediction, dconfidence)

        print('\t  dialog:', dprediction, dconfidence, '\n')
        
        print("Bot : ", actionJar.GetFeedbackDialog(intent, entities, dprediction))
        
        print('\n')
        
except KeyboardInterrupt:
    pass
